In [75]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.stats import differential_entropy,kurtosis
import pandas as pd
from scipy.optimize import minimize

In [76]:
def Majew(gamma, alpha, kappa1, beta, Lambda, sigmav, length = 1000, nb = 1 ):
    #nb : nb de simulations
    # length : taille de la serie temporelle
    Epsilon=np.random.normal(0, sigmav**2, (length,nb))
    v=np.zeros((length,nb))
    m=np.zeros((length,nb))
    
    # We initialize the different functions we will need
    
    p=np.zeros((length,nb))
    r=np.zeros((length,nb))
    p[0] = 200
    # We initialize the price
    
    for t in range(1,length-1):
        
        v[t] = (1-Lambda)*v[t-1] + Lambda*p[t]
        
        m[t] = (1-alpha)*m[t-1] + alpha*(p[t]-p[t-1])
        
        p[t+1] = p[t] + kappa1*(v[t]-p[t]) + beta*np.tanh(gamma*m[t]) + Epsilon[t+1]
        
        r[t+1] = p[t+1] - p[t]
    
    return p,r

In [77]:
def compute_statistics(r, cov=False):
    means = np.mean(r, axis=0)
    std_dev = np.std(r, axis=0)
    entropy = differential_entropy(r, axis = 0)
    kurtosiss = kurtosis(r, axis=0)
    
    q1 = np.percentile(r, 25, axis=0)
    q2 = np.percentile(r, 50, axis=0)
    q3 = np.percentile(r, 75, axis=0)
    
    
    if cov:
        mf = np.vstack([means, std_dev, entropy, kurtosiss, q1, q2, q3])
        W = np.cov(mf)
        
        return np.array([np.mean(means), np.mean(std_dev), np.mean(entropy), np.mean(kurtosiss), np.mean(q1), np.mean(q2), np.mean(q3)]), W
    
    return np.array([np.mean(means), np.mean(std_dev), np.mean(entropy), np.mean(kurtosiss), np.mean(q1), np.mean(q2), np.mean(q3)])



In [78]:
_, r_thild = Majew(0.015,1/7,0.015,36.7,0.2,0.018)
c_thilde = compute_statistics(r_thild)
c_thilde

array([-2.06813223e-01,  1.24019575e+00, -5.20500067e+00,  7.00129506e+01,
       -3.71993475e-04, -6.56648783e-05,  1.77820311e-04])

In [79]:
def cost(ksi):
    _, r = Majew(*ksi, nb = 100)
    cabm, V = compute_statistics(r, cov=True)
    dc = c_thilde - cabm
    W = np.linalg.inv(V)
    D = dc.T @ W @ dc
    return D

In [ ]:
options = {
    'maxiter': 10000,     # Maximum number of iterations        # Precision goal for the function value         # Precision goal for the gradient
    'disp': True          # Display convergence messages
}
init = (0.02,1/9,0.01,32.7,0.3,0.02)
res = minimize(cost, init, method='Nelder-Mead', options=options)

In [ ]:
res

In [94]:
# Define the options for L-BFGS-B
options_lbfgsb = {
    'maxiter': 1000,
    'ftol': 1e-9,
    'gtol': 1e-9,
    'disp': True
}
init_lbfgsb = (0.02,1/9,0.01,32.7,0.3,0.02)
# Perform the optimization using 'L-BFGS-B' method
res_lbfgsb = minimize(cost, init_lbfgsb, method='L-BFGS-B', options=options_lbfgsb)
print("L-BFGS-B Result:")
print(res_lbfgsb)

RUNNING THE L-BFGS-B CODE

           * * *

Machine precision = 2.220D-16
 N =            6     M =           10

At X0         0 variables are exactly at the bounds

At iterate    0    f=  6.93293D+08    |proj g|=  2.69091D+16


 This problem is unconstrained.



At iterate    1    f=  5.90732D+08    |proj g|=  3.45466D+16
L-BFGS-B Result:
At iterate    2    f=  8.07778D+08    |proj g|=  1.82615D+16

           * * *

Tit   = total number of iterations
Tnf   = total number of function evaluations
Tnint = total number of segments explored during Cauchy searches
Skip  = number of BFGS updates skipped
Nact  = number of active bounds at final generalized Cauchy point
Projg = norm of the final projected gradient
F     = final function value

           * * *

   N    Tit     Tnf  Tnint  Skip  Nact     Projg        F
    6      2     28      1     0     0   1.826D+16   8.078D+08
  F =   807777751.00563478     

CONVERGENCE: REL_REDUCTION_OF_F_<=_FACTR*EPSMCH             

  message: CONVERGENCE: REL_REDUCTION_OF_F_<=_FACTR*EPSMCH
  success: True
   status: 0
      fun: 807777751.0056348
        x: [ 2.000e-02  1.111e-01  1.000e-02  3.270e+01  3.000e-01
             2.000e-02]
      nit: 2
      jac: [-1.226e+16 -1.826e+16 -1.507e+16 -1.546e+16 -1.51


   evaluations in the last line search.  Termination
   may possibly be caused by a bad search direction.


In [95]:
# Initial guess
init_slsqp = (0.02,1/9,0.01,32.7,0.3,0.02)

# Define the options for SLSQP
options_slsqp = {
    'maxiter': 1000,
    'ftol': 1e-9,
    'disp': True
}

# Perform the optimization using 'SLSQP' method
res_slsqp = minimize(cost, init_slsqp, method='SLSQP', options=options_slsqp)
print("SLSQP Result:")
print(res_slsqp)


/tmp/ipykernel_1365/1640456040.py:21: RuntimeWarning: overflow encountered in multiply
  p[t+1] = p[t] + kappa1*(v[t]-p[t]) + beta*np.tanh(gamma*m[t]) + Epsilon[t+1]
/tmp/ipykernel_1365/1640456040.py:17: RuntimeWarning: overflow encountered in multiply
  v[t] = (1-Lambda)*v[t-1] + Lambda*p[t]
/tmp/ipykernel_1365/1640456040.py:17: RuntimeWarning: invalid value encountered in add
  v[t] = (1-Lambda)*v[t-1] + Lambda*p[t]
/tmp/ipykernel_1365/1640456040.py:19: RuntimeWarning: overflow encountered in multiply
  m[t] = (1-alpha)*m[t-1] + alpha*(p[t]-p[t-1])
/tmp/ipykernel_1365/1640456040.py:19: RuntimeWarning: invalid value encountered in add
  m[t] = (1-alpha)*m[t-1] + alpha*(p[t]-p[t-1])
/tmp/ipykernel_1365/1640456040.py:21: RuntimeWarning: invalid value encountered in subtract
  p[t+1] = p[t] + kappa1*(v[t]-p[t]) + beta*np.tanh(gamma*m[t]) + Epsilon[t+1]


Iteration limit reached    (Exit mode 9)
            Current function value: nan
            Iterations: 1000
            Function evaluations: 17988
            Gradient evaluations: 1000
SLSQP Result:
 message: Iteration limit reached
 success: False
  status: 9
     fun: nan
       x: [       nan        nan        nan        nan        nan
                  nan]
     nit: 1000
     jac: [       nan        nan        nan        nan        nan
                  nan]
    nfev: 17988
    njev: 1000


In [96]:
# Initial guess
init_tnc = (0.02,1/9,0.01,32.7,0.3,0.02)

# Define the options for TNC
options_tnc = {
    'maxiter': 1000,
    'disp': True,
    'gtol': 1e-9
}

# Perform the optimization using 'TNC' method
res_tnc = minimize(cost, init_tnc, method='TNC', options=options_tnc)
print("TNC Result:")



/tmp/ipykernel_1365/3342169377.py:12: OptimizeWarning: Unknown solver options: maxiter
  res_tnc = minimize(cost, init_tnc, method='TNC', options=options_tnc)
  NIT   NF   F                       GTG
    0    1  9.071449716598027E+08   7.21147732E+32
tnc: fscale = 1.89521e-18
    1   11  7.015524316081628E+08   1.24501358E+33


TNC Result:


tnc: |xn-xn-1] = 1.45215e-08 -> convergence
    2   15  5.797449716158283E+08   1.62143251E+33
tnc: Converged (|x_n-x_(n-1)| ~= 0)


In [97]:
print(cost(res_tnc.x))

808251360.0913874


In [98]:
# Initial guess
init_bfgs = (0.02,1/9,0.01,32.7,0.3,0.02)

# Define the options for BFGS
options_bfgs = {
    'maxiter': 10000,
    'gtol': 1e-9,
    'disp': True
}

# Perform the optimization using 'BFGS' method
res_bfgs = minimize(cost, init_bfgs, method='BFGS', options=options_bfgs)
print("BFGS Result:")
print(res_bfgs)


         Current function value: 630663267.921971
         Iterations: 1
         Function evaluations: 89
         Gradient evaluations: 11
BFGS Result:
  message: Desired error not necessarily achieved due to precision loss.
  success: False
   status: 2
      fun: 630663267.9219714
        x: [ 2.000e-02  1.111e-01  1.000e-02  3.270e+01  3.000e-01
             2.000e-02]
      nit: 1
      jac: [ 1.354e+16  9.438e+15  7.303e+15  8.084e+15  6.096e+15
            -5.925e+15]
 hess_inv: [[ 5.276e-01 -3.342e-01 ... -3.032e-01  2.370e-01]
            [-3.342e-01  7.697e-01 ... -1.006e-01  1.296e-01]
            ...
            [-3.032e-01 -1.006e-01 ...  2.908e+00 -5.507e-01]
            [ 2.370e-01  1.296e-01 ... -5.507e-01  1.116e+00]]
     nfev: 89
     njev: 11


/opt/mamba/lib/python3.12/site-packages/scipy/optimize/_minimize.py:708: OptimizeWarning: Desired error not necessarily achieved due to precision loss.
  res = _minimize_bfgs(fun, x0, args, jac, callback, **options)


In [86]:
def Majew2(gamma, alpha, kappa1, kappa3, beta, Lambda, sigmav):
    
    Epsilon=np.random.normal(0, sigmav**2, (length,nb))
    v=np.zeros((length,nb))
    m=np.zeros((length,nb))
    
    # We initialize the different functions we will need
    
    p=np.zeros((length,nb))
    r=np.zeros((length,nb))
    
    # We initialize the price
    
    for t in range(1,length-1):
        
        v[t] = (1-Lambda)*v[t-1] + Lambda*p[t]
        
        m[t] = (1-alpha)*m[t-1] + alpha*(p[t]-p[t-1])
        
        p[t+1] = p[t] + kappa1*(v[t]-p[t]) + kappa3*(v[t]-p[t]) + beta*np.tanh(gamma*m[t]) + Epsilon[t+1]
        
        r[t+1] = p[t+1] - p[t]
    
    return p,r